In [477]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [479]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME','USE_CASE', 'ASK_AMT','SPECIAL_CONSIDERATIONS'
application_df = application_df.drop(columns = ['EIN', 'NAME','USE_CASE', 'ASK_AMT','SPECIAL_CONSIDERATIONS'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,1,0,1
1,T3,Independent,C2000,Co-operative,1,1-9999,1
2,T5,CompanySponsored,C3000,Association,1,0,0
3,T3,CompanySponsored,C2000,Trust,1,10000-24999,1
4,T3,Independent,C1000,Trust,1,100000-499999,1
...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,Association,1,0,0
34295,T4,CompanySponsored,C3000,Association,1,0,0
34296,T3,CompanySponsored,C2000,Association,1,0,0
34297,T5,Independent,C3000,Association,1,0,1


In [481]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
ORGANIZATION         4
STATUS               2
INCOME_AMT           9
IS_SUCCESSFUL        2
dtype: int64

In [483]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_count=application_df.APPLICATION_TYPE.value_counts()
# the result of above statement is a data series with the number of times each unique value occurs in the data set. 
application_count


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [485]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# This line of code is creating a list of application types that appear less than 400 times in the dataset.
application_types_to_replace = application_count.loc[application_count < 400].index.tolist()
# Replace them with "Other" in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [487]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [489]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count_gt1 = classification_count[classification_count > 1]
print(classification_count_gt1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [491]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# This line of code is creating a list of classifications that appear less than 400 times in the dataset.
classifications_to_replace = classification_count.loc[classification_count < 200].index.tolist()
# Replace them with "Other" in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: count, dtype: int64

In [493]:
# Identify categorical data 
categorical_list = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
categorical_list

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'ORGANIZATION',
 'INCOME_AMT']

In [495]:
#convert the categorical data in numeric using get_dummies()
dummies = pd.get_dummies(application_df)
dummies.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,1,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1,0,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,1,1,False,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,1,1,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [497]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL'].values
X = dummies.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42, stratify=y)

In [499]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Compile, Train and Evaluate the Model¶


In [502]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# (FIRST ATTEMPT - ACCURACY 72.6%)
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 38
hidden_nodes_layer2 = 19

nn = tf.keras.models.Sequential([
    Dense(units=hidden_nodes_layer1, input_dim=input_features_total, activation="relu"),
    BatchNormalization(),
    Dense(units=hidden_nodes_layer2, activation="relu"),
    BatchNormalization(),
    Dense(units=1, activation="relu")
])

# Check the structure of the model
nn.summary()

C:\Users\Nagendra.Sriperumbud\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_68 (Dense)                │ (None, 38)             │         1,444 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 38)             │           152 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 19)             │           741 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 19)             │            76 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 1)              │            20 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,433 (9.50 KB)

 Trainable params: 2,319 (9.06 KB)

 Non-trainable params: 114 (456.00 B)

In [504]:
# Compile the model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [506]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [508]:
# Train the model
#fit_model = nn.fit(X_train_scaled,y_train,epochs=100)
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.5979 - loss: 0.3713 - val_accuracy: 0.6793 - val_loss: 0.2555
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6752 - loss: 0.2509 - val_accuracy: 0.7221 - val_loss: 0.1997
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7141 - loss: 0.2028 - val_accuracy: 0.7176 - val_loss: 0.1959
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7177 - loss: 0.1973 - val_accuracy: 0.7252 - val_loss: 0.1915
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7190 - loss: 0.1938 - val_accuracy: 0.7249 - val_loss: 0.1927
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7222 - loss: 0.1918 - val_accuracy: 0.7278 - val_loss: 0.1900
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7261 - loss: 0.1893 - val_accuracy: 0.7250 - val_loss: 0.1902
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7253 - loss: 0.1903 - val_accu

In [510]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 1ms/step - accuracy: 0.7162 - loss: 0.1890
Loss: 0.18895220756530762, Accuracy: 0.7161807417869568


In [512]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# (2nd ATTEMPT - ACCURACY 71.96%)
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 14
hidden_nodes_layer2 = 7

nn = tf.keras.models.Sequential([
    Dense(units=hidden_nodes_layer1, input_dim=input_features_total, activation="relu"),
    BatchNormalization(),
    Dense(units=hidden_nodes_layer2, activation="relu"),
    BatchNormalization(),
    Dense(units=1, activation="linear")
])

# Check the structure of the model
nn.summary()

C:\Users\Nagendra.Sriperumbud\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_71 (Dense)                │ (None, 14)             │           532 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_48          │ (None, 14)             │            56 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 7)              │           105 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_49          │ (None, 7)              │            28 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_73 (Dense)                │ (None, 1)              │             8 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 729 (2.85 KB)

 Trainable params: 687 (2.68 KB)

 Non-trainable params: 42 (168.00 B)

In [514]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [516]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [518]:
# Train the model
#fit_model = nn.fit(X_train_scaled,y_train,epochs=100)
fit_model = nn.fit(X_train_scaled, y_train, epochs=75, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

Epoch 1/75
686/686 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.4984 - loss: 6.1962 - val_accuracy: 0.5341 - val_loss: 4.6343
Epoch 2/75
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5725 - loss: 4.5122 - val_accuracy: 0.6392 - val_loss: 3.5202
Epoch 3/75
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5752 - loss: 3.9009 - val_accuracy: 0.5027 - val_loss: 3.3729
Epoch 4/75
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5647 - loss: 3.4589 - val_accuracy: 0.6317 - val_loss: 3.0699
Epoch 5/75
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6045 - loss: 2.8233 - val_accuracy: 0.6317 - val_loss: 1.8903
Epoch 6/75
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6226 - loss: 2.1001 - val_accuracy: 0.6383 - val_loss: 1.8454
Epoch 7/75
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6226 - loss: 1.9058 - val_accuracy: 0.6656 - val_loss: 1.6170
Epoch 8/75
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6351 - loss: 1.7068 - val_accuracy: 0.

In [520]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 1ms/step - accuracy: 0.7236 - loss: 0.5716
Loss: 0.5716126561164856, Accuracy: 0.7236151695251465


In [522]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# (3rd ATTEMPT - ACCURACY 72.4%)
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 40
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential([
    Dense(units=hidden_nodes_layer1, input_dim=input_features_total, activation="relu"),
    BatchNormalization(),
    Dense(units=hidden_nodes_layer2, activation="relu"),
    BatchNormalization(),
    Dense(units=hidden_nodes_layer3, activation="relu"),
    BatchNormalization(),
    Dense(units=1, activation="sigmoid")
])

# Check the structure of the model
nn.summary()

C:\Users\Nagendra.Sriperumbud\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_74 (Dense)                │ (None, 40)             │         1,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_50          │ (None, 40)             │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_75 (Dense)                │ (None, 40)             │         1,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_51          │ (None, 40)             │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_76 (Dense)                │ (None, 20)             │           820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_52          │ (None, 20)             │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_77 (Dense)                │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,401 (17.19 KB)

 Trainable params: 4,201 (16.41 KB)

 Non-trainable params: 200 (800.00 B)

In [524]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [526]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [528]:
# Train the model
#fit_model = nn.fit(X_train_scaled,y_train,epochs=100)
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

Epoch 1/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.6661 - loss: 0.6392 - val_accuracy: 0.7236 - val_loss: 0.5692
Epoch 2/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7271 - loss: 0.5625 - val_accuracy: 0.7225 - val_loss: 0.5679
Epoch 3/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7318 - loss: 0.5580 - val_accuracy: 0.7241 - val_loss: 0.5595
Epoch 4/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7298 - loss: 0.5572 - val_accuracy: 0.7252 - val_loss: 0.5615
Epoch 5/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7290 - loss: 0.5543 - val_accuracy: 0.7250 - val_loss: 0.5645
Epoch 6/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7281 - loss: 0.5597 - val_accuracy: 0.7267 - val_loss: 0.5633
Epoch 7/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7317 - loss: 0.5559 - val_accuracy: 0.7259 - val_loss: 0.5608
Epoch 8/50
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7283 - loss: 0.5541 - val_accuracy: 0.

In [530]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 1ms/step - accuracy: 0.7224 - loss: 0.5587
Loss: 0.5587441921234131, Accuracy: 0.722449004650116


In [532]:
# Export the model to HDF5 file
nn.save("AlphabetSoupCharity_Optimisation.h5")
nn.save('AlphabetSoupCharity_Optimisation.keras')
